# Plot driver standings in a heatmap

Plot the points for each driven in each race of a given season in a heatmap, as
https://public.tableau.com/app/profile/mateusz.karmalski/viz/F1ResultsTracker2022


In [1]:
import pandas as pd
import plotly.express as px
from plotly.io import show

from fastf1.ergast import Ergast

First, we load the results for season 2022.



In [2]:
ergast = Ergast()
races = ergast.get_race_schedule(2022)  # Races in year 2022
results = []

# For each race in the season
for rnd, race in races['raceName'].items():

    # Get results. Note that we use the round no. + 1, because the round no.
    # starts from one (1) instead of zero (0)
    temp = ergast.get_race_results(season=2022, round=rnd + 1)
    temp = temp.content[0]

    # If there is a sprint, get the results as well
    sprint = ergast.get_sprint_results(season=2022, round=rnd + 1)
    if sprint.content and sprint.description['round'][0] == rnd + 1:
        temp = pd.merge(temp, sprint.content[0], on='driverCode', how='left')
        # Add sprint points and race points to get the total
        temp['points'] = temp['points_x'] + temp['points_y']
        temp.drop(columns=['points_x', 'points_y'], inplace=True)

    # Add round no. and grand prix name
    temp['round'] = rnd + 1
    temp['race'] = race.removesuffix(' Grand Prix')
    temp = temp[['round', 'race', 'driverCode', 'points']]  # Keep useful cols.
    results.append(temp)

# Append all races into a single dataframe
results = pd.concat(results)
races = results['race'].drop_duplicates()

req         WARNING 	DEFAULT CACHE ENABLED! (600.06 MB) /home/rylei/.cache/fastf1


Then we “reshape” the results to a wide table, where each row represents a
driver and each column refers to a race, and the cell value is the points.



In [3]:
results = results.pivot(index='driverCode', columns='round', values='points')
# Here we have a 22-by-22 matrix (22 races and 22 drivers, incl. DEV and HUL)

# Rank the drivers by their total points
results['total_points'] = results.sum(axis=1)
results = results.sort_values(by='total_points', ascending=False)
results.drop(columns='total_points', inplace=True)

# Use race name, instead of round no., as column names
results.columns = races

In [6]:
results

race,Bahrain,Saudi Arabian,Australian,Emilia Romagna,Miami,Spanish,Monaco,Azerbaijan,Canadian,British,...,Hungarian,Belgian,Dutch,Italian,Singapore,Japanese,United States,Mexico City,São Paulo,Abu Dhabi
driverCode,,,,,,,,,,,,,,,,,,,,,
VER,0.0,25.0,0.0,34.0,26.0,25.0,15.0,25.0,25.0,6.0,...,25.0,26.0,26.0,25.0,6.0,25.0,25.0,25.0,13.0,25.0
LEC,26.0,19.0,26.0,15.0,18.0,0.0,12.0,0.0,10.0,12.0,...,8.0,8.0,15.0,18.0,18.0,15.0,15.0,8.0,15.0,18.0
PER,0.0,12.0,18.0,24.0,12.0,19.0,25.0,19.0,0.0,18.0,...,10.0,18.0,10.0,9.0,25.0,18.0,12.0,15.0,10.0,15.0
RUS,12.0,10.0,15.0,12.0,10.0,15.0,10.0,15.0,12.0,0.0,...,15.0,12.0,18.0,15.0,0.0,4.0,11.0,13.0,34.0,10.0
SAI,18.0,15.0,0.0,5.0,15.0,12.0,18.0,0.0,19.0,25.0,...,12.0,15.0,4.0,12.0,15.0,0.0,0.0,10.0,22.0,12.0
HAM,15.0,1.0,12.0,0.0,8.0,10.0,4.0,12.0,15.0,16.0,...,19.0,0.0,12.0,10.0,2.0,10.0,18.0,18.0,24.0,0.0
NOR,0.0,6.0,10.0,19.0,0.0,4.0,9.0,2.0,0.0,8.0,...,6.0,0.0,6.0,6.0,12.0,1.0,8.0,2.0,2.0,9.0
OCO,6.0,8.0,6.0,0.0,4.0,6.0,0.0,1.0,8.0,0.0,...,2.0,6.0,2.0,0.0,0.0,12.0,0.0,4.0,4.0,6.0
ALO,2.0,0.0,0.0,0.0,0.0,2.0,6.0,6.0,2.0,10.0,...,4.0,10.0,8.0,0.0,0.0,6.0,6.0,0.0,10.0,0.0


The final step is to plot a heatmap using plotly



In [5]:
fig = px.imshow(
    results,
    text_auto=True,
    aspect='auto',  # Automatically adjust the aspect ratio
    color_continuous_scale=[[0,    'rgb(198, 219, 239)'],  # Blue scale
                            [0.25, 'rgb(107, 174, 214)'],
                            [0.5,  'rgb(33,  113, 181)'],
                            [0.75, 'rgb(8,   81,  156)'],
                            [1,    'rgb(8,   48,  107)']],
    labels={'x': 'Race',
            'y': 'Driver',
            'color': 'Points'}       # Change hover texts
)
fig.update_xaxes(title_text='')      # Remove axis titles
fig.update_yaxes(title_text='')
fig.update_yaxes(tickmode='linear')  # Show all ticks, i.e. driver names
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey',
                 showline=False,
                 tickson='boundaries')              # Show horizontal grid only
fig.update_xaxes(showgrid=False, showline=False)    # And remove vertical grid
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')     # White background
fig.update_layout(coloraxis_showscale=False)        # Remove legend
fig.update_layout(xaxis=dict(side='top'))           # x-axis on top
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))  # Remove border margins
fig
#show(fig)